Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [330]:
from random import choices,random,randint
import numpy as np
import lab9_lib
from pprint import pprint


## PARAMETERS INITIALIZATION & PROBLEM GENERATION

In [331]:
PROBLEM_DIM = 10

GENES_PER_LOCUS = 1
GENOME_LENGHT=1000*GENES_PER_LOCUS
POP_DIM= 500
OFFSPRING_SIZE = 200
N_GENERATIONS = 1000
N_GENS_WO_IMPROVEMENT_EXTINCTION = 5
MUTATION_PROB = 0.2

fitness = lab9_lib.make_problem(PROBLEM_DIM)

# for n in range(10):
#     ind = choices([0, 1], k=50)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)

## INDIVIDUAL CLASS

In [332]:
class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)

    def get_genome(self):
        return self.genome

    def get_fitness(self):
        return self.fitness 

    def set_genome_update_fitness(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)

## GA

In [333]:
def one_cut_xover(g1, g2):
    cut = randint(0, GENOME_LENGHT)
    joined = g1.get_genome()[:cut] + g2.get_genome()[cut:]
    return Individual(joined)

def two_cut_xover(g1, g2):
    cut1 = randint(0, GENOME_LENGHT)
    cut2 = randint(0, GENOME_LENGHT)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    joined = g1.get_genome()[:cut1] + g2.get_genome()[cut1:cut2] + g1.get_genome()[cut2:]
    return Individual(joined)

def tournament_selection(pop, size=2):
    c_ind = choices(range(len(pop)), k=size)
    selected_individuals = [pop[i] for i in c_ind]
    # print("sel", selected_individuals)
    return sorted(selected_individuals, key=lambda i:i.get_fitness(), reverse=True)[0]


def mutation(g):
    gene_to_mutate = randint(0, GENOME_LENGHT - 1)
    mutated=g.get_genome()[:gene_to_mutate]+ [(1 - g.get_genome()[gene_to_mutate])]+ g.get_genome()[gene_to_mutate + 1 :]
    # print("mut", mutated)
    return Individual(mutated)

def extinction(pop):

    to_remove= np.random.choice(pop, size=POP_DIM//3,replace=False)
    print("to_remove", to_remove)
    for i in to_remove:
        pop.remove(i)
    pop+= [Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM//5)]

    return pop

In [334]:
countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

pop=[Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM)]

best=max(pop, key=lambda i: i.get_fitness())

for gen in range(N_GENERATIONS):

    if countdown_to_extinction == 0:
        print("Extinction!")
        pop=extinction(pop)
        countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

    for off in range(OFFSPRING_SIZE):

        if random() < MUTATION_PROB:
            p=tournament_selection(pop)
            offspring = mutation(p)
        else:
            offspring = two_cut_xover(tournament_selection(pop), tournament_selection(pop))
            # print("off_fitness", offspring.get_fitness())
        pop.append(offspring)

    pop=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:POP_DIM]


    if pop[0].get_fitness() == best.get_fitness():
        countdown_to_extinction-=1
    else:
        countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

    best=pop[0]

    print(f"Generation {gen}: {best.get_fitness():.2%}")
print(f"Best individual: {''.join(str(g) for g in best.get_genome())}")
pprint(fitness.calls)


Generation 0: 27.39%
Generation 1: 27.39%
Generation 2: 27.39%
Generation 3: 27.39%
Generation 4: 27.39%
Extinction!
to_remove [<__main__.Individual object at 0x0000021D6F90C3E0>
Generation 5: 27.39%
Generation 6: 33.05%
Generation 7: 33.05%
Generation 8: 33.05%
Generation 9: 33.05%
Generation 10: 33.05%
Generation 11: 33.05%
Extinction!
to_remove [<__main__.Individual object at 0x0000021D59153950>
Generation 12: 33.05%
Generation 13: 33.05%
Generation 14: 33.05%
Generation 15: 33.05%
Generation 16: 33.05%
Extinction!
to_remove [<__main__.Individual object at 0x0000021D47DEA930>
Generation 17: 27.40%
Generation 18: 27.40%
Generation 19: 27.40%
Generation 20: 27.40%
Generation 21: 27.40%
Generation 22: 27.40%
Generation 23: 27.40%
Generation 24: 27.40%
Generation 25: 27.40%
Extinction!
to_remove [<__main__.Individual object at 0x0000021D1EB7E210>
Generation 26: 33.05%
Generation 27: 33.05%
Generation 28: 33.05%
Generation 29: 33.05%
Generation 30: 33.05%
Generation 31: 33.05%
Extinction

KeyboardInterrupt: 